In [1]:
import pandas as pd
df = pd.read_csv(r"D:\Cleaned_data.csv")
print(df.head())

   Unnamed: 0  target          id                          date      flag  \
0           0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1           1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2           2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3           3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4           4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   
3          ElleCTF    my whole body feels itchy and like its on fire    
4           Karoli  @nationwideclass no, it's not behaving at all....   

                                     stemmed_content  
0  switchfoot http twitpic com zl awww bumm

In [2]:
from sklearn.model_selection import train_test_split
df.shape

(1600000, 8)

In [3]:
#To know number of null values in each column
df.isnull().sum()

Unnamed: 0           0
target               0
id                   0
date                 0
flag                 0
user                 0
text                 0
stemmed_content    495
dtype: int64

In [4]:
df.dropna(subset=["stemmed_content"],inplace=True)

In [5]:
#slicing
df=df[::8]
df.shape

(199939, 8)

In [6]:
#after using dropna on stemmed content to remove those rows where stemmed content column has null values
df.isnull().sum()

Unnamed: 0         0
target             0
id                 0
date               0
flag               0
user               0
text               0
stemmed_content    0
dtype: int64

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
X = df['stemmed_content']
y = df['target']


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X) 
X_seq = tokenizer.texts_to_sequences(X) 

In [10]:
#tokenized representations of your text data
print("\nTokenized form of text data is :")
for i in X_seq:
 print(i)



Tokenized form of text data is :
[17112, 9, 82, 21, 10952, 369, 1078, 3036, 12, 614, 7634, 1724, 3]
[22674, 346, 615]
[2173, 986, 1534, 134, 1579, 33, 635, 22675, 4615, 458]
[37279, 283, 45]
[167, 47, 1351, 45, 586, 7635, 42, 5, 516]
[24, 26, 6, 2, 10, 12, 253, 45, 747, 219]
[1290, 50, 17113, 37280, 334]
[14111, 70, 203]
[69, 3, 57, 99, 39, 45]
[24, 70, 78, 6621, 568, 3699, 2360, 1420]
[47, 96, 70, 63, 469, 23, 45, 28]
[122, 893, 19]
[12315, 2032, 2307, 1, 1666, 186, 713, 217, 2786, 4191, 152, 376, 41, 927, 32]
[37281, 324, 716, 628, 2224, 16, 11, 1041]
[37282, 61, 161, 17114, 37283, 24, 61]
[684, 37284, 748, 1175]
[240]
[17, 3423, 677]
[3112, 230, 41, 10, 123, 134]
[1176, 442]
[411, 5912, 377, 264, 2, 1, 26, 153, 4, 411, 64, 60, 174]
[22676, 1041, 14]
[37285, 268, 64, 4290, 1020]
[263, 128, 1986, 299, 769, 1, 168, 1400, 10, 1364]
[4192, 4, 90]
[226, 759, 5710, 10, 44, 239, 12, 81, 47]
[37286]
[2248, 12316, 379, 299, 113, 170, 702, 247, 1017]
[17, 40, 132, 467, 92, 296, 3490, 132, 222

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[220, 1248, 164, 16, 2198, 101, 24, 290]
[2, 148, 271, 1396, 255, 934]
[176, 2682, 7924, 58, 756]
[16, 3027]
[119441, 9176, 326, 2040, 417, 220]
[20093, 13053, 18, 19]
[7580, 2254, 12, 381, 895, 23, 65, 593]
[119442, 217, 6, 780, 198, 121, 246, 142, 215, 137]
[16, 556, 4, 75, 78]
[15915, 320, 2962, 18, 235, 11, 18]
[119443, 4, 195, 1624, 38, 24, 756, 49]
[119444, 294, 518, 821, 397, 397, 10, 17506, 204, 2100, 2002, 8, 1643, 613, 8]
[199, 7113, 2133, 119445, 120, 27, 120, 3432, 18, 11002, 463, 16593, 11002]
[119446, 29, 192, 703, 15, 16, 39, 330, 30, 23, 1594]
[447, 119447, 4319, 348, 160, 17, 15, 126, 140, 745, 292, 322, 15, 224, 12555, 49, 93, 7, 119448]
[119449, 49, 214, 4, 19, 390, 79, 31, 119450, 49, 96, 2, 36, 45, 14]
[30937, 119451, 73, 286, 286]
[25, 153, 1587, 12019, 176, 4438, 40, 299, 769, 3310, 21, 584, 165, 590, 1511, 2068, 509, 9176]
[119452, 7612, 2908, 3563, 8328, 1316, 959, 3563, 36490, 470, 26852, 1370, 3325, 118, 14]
[119453, 119454, 13451, 119455, 146, 6, 30, 313]
[2

In [11]:
#to check number of rows in our df
count=0
for i in X_seq:
 count=count+1
print("Number of rows are :" ,count)


Number of rows are : 199939


In [12]:
#longest sequence in tokenized text data
max_len = max([len(leni) for leni in X_seq])
print("Longest sequence in X_seq is of length : " ,max_len)

Longest sequence in X_seq is of length :  40


In [13]:
#pad_sequences function to ensure that all sequences in X_seq have the same length
X_padded = pad_sequences(X_seq, maxlen=40) 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)
# Adding 1 to store dimensions for words for which no pretrained word embedding
vocab_length = len(tokenizer.word_index) + 1
vocab_length

128825

In [14]:
from numpy import asarray #is used to convert input into an array.
from numpy import zeros  #zeros is used to create an array filled with zeros.

embeddings_dictionary = dict() #This dictionary will be used to store word embeddings.
glove_file = open(r"C:\Users\HP\OneDrive\Desktop\Ayushi\glove.6B.100d.txt", encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [15]:
embedding_matrix = zeros((vocab_length, 100))
#. For each word, it gets both the word itself (word) and its corresponding index (index) in the tokenizer's word index.
for word, index in tokenizer.word_index.items():
 embedding_vector = embeddings_dictionary.get(word)
 if embedding_vector is not None:
  embedding_matrix[index] = embedding_vector

In [16]:
# returns the shape of training data matrix X_train
X_train.shape

(159951, 40)

In [17]:
#returns the shape of testing data matrix X_test
X_test.shape

(39988, 40)

In [18]:
#returns the unique number of wordsin the vocabulary
len(tokenizer.word_index)


128824

In [19]:
y_train.shape

(159951,)

In [20]:
y_test.shape

(39988,)

In [21]:
#LSTM model
model = Sequential()
model.add(Embedding(128825, 100, input_length=40))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 100)           12882500  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 12999877 (49.59 MB)
Trainable params: 12999877 (49.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [23]:
model_history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_test, y_test))



Epoch 1/6
1250/1250 [==============================] - 314s 247ms/step - loss: 0.5058 - accuracy: 0.7502 - val_loss: 0.4804 - val_accuracy: 0.7661
Epoch 2/6
1250/1250 [==============================] - 310s 248ms/step - loss: 0.3733 - accuracy: 0.8348 - val_loss: 0.5139 - val_accuracy: 0.7533
Epoch 3/6
1250/1250 [==============================] - 314s 251ms/step - loss: 0.2575 - accuracy: 0.8910 - val_loss: 0.5904 - val_accuracy: 0.7455
Epoch 4/6
1250/1250 [==============================] - 306s 245ms/step - loss: 0.1892 - accuracy: 0.9191 - val_loss: 0.6911 - val_accuracy: 0.7337
Epoch 5/6
1250/1250 [==============================] - 306s 245ms/step - loss: 0.1478 - accuracy: 0.9362 - val_loss: 0.8408 - val_accuracy: 0.7328
Epoch 6/6
1250/1250 [==============================] - 320s 256ms/step - loss: 0.1199 - accuracy: 0.9486 - val_loss: 0.9469 - val_accuracy: 0.7264


In [24]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


1250/1250 [==============================] - 25s 20ms/step - loss: 0.9469 - accuracy: 0.7264
Test loss: 0.9468969702720642
Test accuracy: 0.7264178991317749
